# Домашнее задание  № 5. Матричные разложения/Тематическое моделирование

### Задание № 1 (4 балла)

Попробуйте матричные разложения с 4 классификаторами - SGDClassifier, KNeighborsClassifier,  RandomForest, ExtraTreesClassifier (про него подробнее почитайте в документации, он похож на RF). Используйте и NMF и SVD. Сравните результаты на кросс-валидации и выберите лучшее сочетание.

В итоге у вас должно получиться, как минимум 8 моделей (два разложения на каждый классификатор). Используйте 1 и те же параметры кросс-валидации. Параметры векторизации, параметры K в матричных разложениях, параметры классификаторов могут быть разными между экспериментами.

Можете взять поменьше данных, если все будет обучаться слишком долго (не ставьте параметр K слишком большим в NMF, иначе точно будет слишком долго)

**Необходимые импорты**

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np

from pymorphy2 import MorphAnalyzer
from string import punctuation
from razdel import tokenize as razdel_tokenize

from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, StratifiedKFold

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier

morph = MorphAnalyzer()

**Вспомогательные функции по нормализации текста и создании таблиц с метриками оценки модели**

In [3]:
def normalize(text):
    global morph
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20]
    normalized_text = [morph.parse(word)[0].normal_form for word in normalized_text]
    return ' '.join(normalized_text)

def eval_table(X, y, pipeline, N=6):
    labels = list(set(y))
    
    fold_metrics = pd.DataFrame(index=labels)
    errors = np.zeros((len(labels), len(labels)))
    
    kfold = StratifiedKFold(n_splits=N, shuffle=True, )
    
    for i, (train_index, test_index) in enumerate(kfold.split(X, y)):
        pipeline.fit(X[train_index], y[train_index])
        preds = pipeline.predict(X[test_index])
        
        fold_metrics[f'precision_{i}'] = precision_score(y[test_index], preds, labels=labels, average=None)
        fold_metrics[f'recall_{i}'] = recall_score(y[test_index], preds, labels=labels, average=None)
        fold_metrics[f'f1_{i}'] = f1_score(y[test_index], preds, labels=labels, average=None)
        errors += confusion_matrix(y[test_index], preds, labels=labels, normalize='true')
    
    result = pd.DataFrame(index=labels)
    result['precision'] = fold_metrics[[f'precision_{i}' for i in range(N)]].mean(axis=1).round(2)
    result['precision_std'] = fold_metrics[[f'precision_{i}' for i in range(N)]].std(axis=1).round(2)
    
    result['recall'] = fold_metrics[[f'recall_{i}' for i in range(N)]].mean(axis=1).round(2)
    result['recall_std'] = fold_metrics[[f'recall_{i}' for i in range(N)]].std(axis=1).round(2)
    
    result['f1'] = fold_metrics[[f'f1_{i}' for i in range(N)]].mean(axis=1).round(2)
    result['f1_std'] = fold_metrics[[f'f1_{i}' for i in range(N)]].std(axis=1).round(2)
    
    result.loc['mean'] = result.mean().round(2)
    errors /= N
    
    return result, errors

**Подгрузка и нормализация данных**

In [7]:
data = pd.read_csv('avito_category_classification.csv')[:1000]
data['description_norm'] = data['description'].apply(normalize)

**Создаём список с нужными пайплайнами при помощи генератора**

! Векторизация между экспериментами с зафиксированными параметрами. Также модели зафиксированы через random_state, дополнительные параметры не добавлялись.

In [8]:
def models_and_decompose(models: list) -> list:
    global decompose
    for model in models:
        for decomposition in decompose:
            yield Pipeline([
                    ('bow', CountVectorizer(tokenizer=lambda x: x.split(), ngram_range=(1,2), min_df=5, max_df=0.4)),
                    ('svd', decomposition),
                    ('clf', model)
                ])

In [9]:
models = [RandomForestClassifier(random_state=42),
         SGDClassifier(random_state=42),
         ExtraTreesClassifier(random_state=42),
         KNeighborsClassifier()]
decompose = [NMF(25), TruncatedSVD(250)]
pipes = list(models_and_decompose(models))

**Обучаем и оцениваем модели**

! Для сравнения создаём общую табличку с наименованием модели и декомпозитора, среднего значения Ф-меры, лучше всего определяемой категории и Ф-меры для неё.

In [10]:
from tqdm.notebook import tqdm

RESULTS = []
ERRORS = []

comparison_res = {'Model': [], 'Decomposition': [], 'Mean_F1': [], 
                  'Best_category_recognised': [], 'F1_for_best_category': []}

for pipe in tqdm(pipes):
    res, err = eval_table(data['description_norm'], data['category_name'], pipe)
    RESULTS.append(res)
    ERRORS.append(err)
    model = str(pipe.steps[2][1])
    decompose = str(pipe.steps[1][1])
    comparison_res['Model'].append(model[:model.index('(')])
    comparison_res['Decomposition'].append(decompose[:decompose.index('(')])
    comparison_res['Mean_F1'].append(res.f1['mean'])
    comparison_res['Best_category_recognised'].append(res.f1.idxmax())
    comparison_res['F1_for_best_category'].append(res.f1.max())

  0%|          | 0/8 [00:00<?, ?it/s]

In [11]:
for_comparison = pd.DataFrame(comparison_res)
for_comparison

,Model,Decomposition,Mean_F1,Best_category_recognised,F1_for_best_category
0,RandomForestClassifier,NMF,0.41,Квартиры,0.91
1,RandomForestClassifier,TruncatedSVD,0.29,Квартиры,0.83
2,SGDClassifier,NMF,0.37,Квартиры,0.93
3,SGDClassifier,TruncatedSVD,0.52,Квартиры,0.93
4,ExtraTreesClassifier,NMF,0.41,Квартиры,0.91
5,ExtraTreesClassifier,TruncatedSVD,0.22,Квартиры,0.54
6,KNeighborsClassifier,NMF,0.32,Квартиры,0.84
7,KNeighborsClassifier,TruncatedSVD,0.27,Квартиры,0.61


**Вывод:**

лучше всего согласно метрикам отработала модель *ExtraTreesClassifier* и декомпозитор *NMF*. Лучше всего распознается класс *Квартиры*.

### Задание № 2 (6 баллов)

В Gensim тоже можно добавить нграммы и tf-idf. Постройте 1 модель без них (как в семинаре) и еще 3 модели (1 с нграммами, 1 с tfidf и 1 с нграммами и с tfidf). Сравните качество с помощью метрик (перплексия, когерентность) и на глаз. Определите лучшую модель. Для каждой модели выберите 1 самую красивую на ваш взгляд тему.

Используйте данные википедии из семинара. Можете взять поменьше данных, если все обучается долго.

Важное требование - получившиеся модели не должны быть совсем плохими. Если хороших тем не получается, попробуйте настроить гиперпараметры, отфильтровать словарь по-другому. 

In [72]:
import gensim

**Готовим тексты**

In [12]:
texts = open('wiki_data.txt', encoding='utf-8').read().splitlines()[:1500]
texts = [normalize(text) for text in tqdm(texts)]

  0%|          | 0/1500 [00:00<?, ?it/s]

**Создаем lda модели с разными настройками**

Они будут храниться в словаре `MODEL_STORAGE`

In [42]:
def compile_lda(texts: list, ngram:bool =False, tfidf:bool =False):
    if ngram:
        texts = [text.split() for text in texts]
        ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.4) # threshold можно подбирать
        p = gensim.models.phrases.Phraser(ph)
        ngrammed_texts = p[texts]
        dictionary = gensim.corpora.Dictionary(ngrammed_texts)
    else:
        dictionary = gensim.corpora.Dictionary((text.split() for text in texts))
        
    dictionary.filter_extremes(no_above=0.1, no_below=10)
    dictionary.compactify()
    
    if ngram:
        corpus = [dictionary.doc2bow(text) for text in ngrammed_texts]
    else:
        corpus = [dictionary.doc2bow(text.split()) for text in texts]
    
    if tfidf:
        tfidf = gensim.models.TfidfModel(corpus, id2word=dictionary, )
        corpus = tfidf[corpus]
    
    lda = gensim.models.LdaMulticore(corpus, 
                                 100, # количество тем
                                 alpha='asymmetric',
                                 id2word=dictionary, 
                                 passes=10) 
    
    return dictionary, corpus, lda

In [45]:
MODEL_STORAGE = {'-': compile_lda(texts),
                 'n-gram': compile_lda(texts, ngram=True),
                 'tf-idf': compile_lda(texts, tfidf=True),
                 'n-gram + tf-idf': compile_lda(texts, ngram=True, tfidf=True)
                }

**Выведем слова, характеризующие 5 топиков каждой модели**

In [55]:
for k in MODEL_STORAGE:
    print(f'Themes for {k} model')
    for i, topic in MODEL_STORAGE[k][2].print_topics(num_topics=5):
        words = ' - '.join(re.findall(r'"(\w+)"', topic))
        print(f"Тема {i}: {words}")
    print()

Themes for - model
Тема 99: песня - композитор - опера - музыкальный - граф - музыка - замок - мой - голос
Тема 98: армия - июнь - оборона - собрание - июль - сила - направление - подразделение - корпус - батальон
Тема 2: ирландия - матч - зелёный - 1968 - 1956 - ирландский - 1960 - 1972 - нация - австралия
Тема 1: театр - фильм - произведение - барселона - роман - премия - президент - семья - писатель - роль
Тема 0: люксембург - турнир - де - пара - 2006 - начинать - парижский - оскар - уже - непосредственный

Themes for n-gram model
Тема 99: вино - украина - производство - семья - регион - украинский - площадь - можно - страна - коммуна
Тема 98: кладбище - г - дом - участок - городской - через - установить - семья - аэропорт - день
Тема 2: зимний_олимпийский - спортсмен - ни_один - не_завоевать - история_но - медаль_сборная - страна_представлять - принимать_участие - медаль - тот_число
Тема 1: театр - король - сын - франция - жизнь - французский - сергей - взять - я - через
Тема 0: б

Кажется, что лучше всего из представленных определились топики для самой простой модели. Для моделей с tf-idf почему-то топики повторяются

**Визуализируем разбиение на топики для лучшей модели**

In [71]:
from pyLDAvis.gensim import prepare
pyLDAvis.enable_notebook()

In [73]:
lda = gensim.models.LdaModel(corpus, 200, id2word=dictionary, passes=5)

In [80]:
prepare(*MODEL_STORAGE['-'][::-1])

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
16    -0.009407  0.069944       1        1  4.376247
73     0.199303 -0.159419       2        1  2.849588
75     0.053620  0.188990       3        1  2.552290
35    -0.002249  0.055412       4        1  2.252852
24    -0.073451  0.033287       5        1  2.181795
...         ...       ...     ...      ...       ...
64    -0.083201 -0.008483      96        1  0.298188
79     0.049016  0.055156      97        1  0.252116
72     0.052080  0.020814      98        1  0.244926
74     0.005226 -0.030169      99        1  0.181267
87     0.078524  0.008026     100        1  0.167756

[100 rows x 5 columns], topic_info=          Term        Freq       Total  Category  logprob  loglift
549       уезд  289.000000  289.000000   Default  30.0000  30.0000
61        село  337.000000  337.000000   Default  29.0000  29.0000
52     посёлок  225.000000  225.000000   Default  28.0000  28.0000
99      остров  384.000000  384.000000   Default  27.0000  27.0000
1244  компания  330.000000  330.000000   Default  26.0000  26.0000
...        ...         ...         ...       ...      ...      ...
225   называть    1.916401  140.481958  Topic100  -4.9028   2.0958
377      длина    1.799584  179.657741  Topic100  -4.9657   1.7869
21      данные    1.758008  139.917324  Topic100  -4.9890   2.0135
2314     менее    1.757450   93.636984  Topic100  -4.9894   2.4149
551        ход    1.757406  153.975876  Topic100  -4.9894   1.9175

[5096 rows x 6 columns], token_table=      Topic      Freq Term
term                      
553       2  0.387220    0
553       4  0.029786    0
553      21  0.059572    0
553      25  0.148931    0
553      30  0.059572    0
...     ...       ...  ...
421      88  0.011982    №
421      89  0.017973    №
421      90  0.005991    №
421      91  0.011982    №
421      96  0.023964    №

[38124 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[17, 74, 76, 36, 25, 63, 8, 16, 7, 19, 32, 40, 96, 81, 89, 67, 58, 64, 2, 20, 69, 99, 15, 48, 3, 86, 23, 83, 21, 37, 62, 1, 68, 24, 12, 98, 46, 70, 71, 59, 6, 91, 55, 13, 45, 94, 49, 5, 78, 54, 4, 51, 30, 38, 82, 50, 14, 97, 9, 42, 11, 47, 10, 31, 72, 90, 87, 57, 77, 95, 53, 100, 44, 34, 22, 85, 39, 41, 35, 79, 56, 52, 28, 60, 93, 18, 92, 26, 29, 33, 43, 61, 84, 27, 66, 65, 80, 73, 75, 88])

Хороший перфоманс модели заключается, например, в том, что выделились одна более общая тема **СПОРТ** (топик 2) и внутри неё тема **ОЛИМПИАДА** (топик 18), впрочем топики 51, 54 и 59 и другие в правом нижнем углу также посвящены спорту и расположены отдельно от рассмотренного выше кластера. Видно, что как внутри темы "спорт", так и уже в теме "олимпиада" можно найти подтемы.

**Оценим перплексию для каждой модели**

In [84]:
for k in MODEL_STORAGE:
    perplexity = np.exp2(-MODEL_STORAGE[k][2].log_perplexity(MODEL_STORAGE[k][1]))
    print(f'Perplexity value for model {k}\t{perplexity}')

Perplexity value for model -	260.94256875957836
Perplexity value for model n-gram	317.38611121907894
Perplexity value for model tf-idf	11006.55692898773
Perplexity value for model n-gram + tf-idf	15862.22490135699


Как видно, на глаз была правильно определена лучшая модель, поскольку для нее самый низкий уровень перплексии

**Когерентность**

In [88]:
texts = [text.split() for text in texts]
ph = gensim.models.Phrases(texts, scoring='npmi', threshold=0.4) # threshold можно подбирать
p = gensim.models.phrases.Phraser(ph)
ngrammed_texts = p[texts]

In [89]:
def coherence_values():
    global texts, ngrammed_texts
    for k, (d, c, m) in MODEL_STORAGE.items():
        topics = []
        for topic_id, topic in m.show_topics(num_topics=100, formatted=False):
            topic = [word for word, _ in topic]
            topics.append(topic)
        if 'n-gram' in k:
            coherence_model_lda = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=ngrammed_texts, 
                                                   dictionary=d, coherence='c_v')
        else:
            coherence_model_lda = gensim.models.CoherenceModel(topics=topics, 
                                                   texts=texts, 
                                                   dictionary=d, coherence='c_v')
        yield f'Coherence metric for model {k} - {coherence_model_lda.get_coherence()}'

In [90]:
print('\n'.join(list(coherence_values())))

Coherence metric for model - - 0.47251400388334086
Coherence metric for model n-gram - 0.445537697630991
Coherence metric for model tf-idf - 0.41951977420345693
Coherence metric for model n-gram + tf-idf - 0.44895284774112004


По метрикам когерентности модель без н-грам и тф-идф также превосходит другие